In [1]:
# setup db
from pymongo import MongoClient
import pandas  as pd
import numpy as np


In [11]:
data = pd.read_csv("data/Form.csv")

In [12]:
df = data.iloc[0:,[1,2,3]]

In [25]:
client = MongoClient("localhost",
                     port=27018,
                     username='root',
                     password='example',
                     authSource='admin')

In [26]:
db = client['Infomations']

In [27]:
collections = db['User']

In [221]:
pipeline = [
        {
          '$lookup':
             {
               'from': "Transactions",
               'localField': "uid",
               'foreignField': "uid",
               'as': "combined_documents"
             }
        },
        {
            '$match': { 'combined_documents.event': "like" }
        }
]

In [7]:
pipeline = [
    {
        '$lookup': {
            'from': "Transactions",
            'localField': "uid",
            'foreignField': "uid",
            'as': "combined_documents"
        }
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$unwind': "$combined_documents"
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$group': {
            '_id': "$_id",
            'name': { '$first': "$name" },
            'gender': { '$first': "$gender" },
            'age': { '$first': "$age" },
            'content_list': {
                '$push': "$combined_documents.Content"
            }
        }
    }
]

In [28]:
pipeline = [
    {
        '$lookup': {
            'from': "Transactions",
            'localField': "uid",
            'foreignField': "uid",
            'as': "combined_documents"
        }
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$unwind': "$combined_documents"
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$group': {
            '_id': "$_id",
            'name': { '$first': "$name" },
            'Gender': { '$first': "$gender" },
            'Age': { '$first': "$age" },
            'Store': {
                '$addToSet': "$combined_documents.Content"
            }
        }
    }
]


In [33]:
cl = list(collections.aggregate(pipeline))

In [30]:
from pprint import pprint

In [34]:
for x in cl:

    pprint([doc['event'] for doc in x['combined_documents']])
    pprint([doc['Content'] for doc in x['combined_documents']])
    pprint(x['name'])
    pprint(x['gender'])
    pprint(x['age'])


    print('')
        # print(x['combined_documents'][0]['event'])
        # print(x['combined_documents'][0]['Content'])


KeyError: 'combined_documents'

In [32]:
import json

In [36]:
for x in cl:
    pprint(x)

{'Age': 21,
 'Gender': 'Male',
 'Store': ['Nap x Warin'],
 '_id': ObjectId('63d6163840d228b1ae426045'),
 'name': 'notechu'}
{'Age': 21,
 'Gender': 'Male',
 'Store': ['Nap x Warin', "NAP's Coffee & Roasters"],
 '_id': ObjectId('63d3d51eb0cd665042d3f8e0'),
 'name': 'tong1'}


In [37]:
for x in cl:
    print(x)

{'_id': ObjectId('63d6163840d228b1ae426045'), 'name': 'notechu', 'Gender': 'Male', 'Age': 21, 'Store': ['Nap x Warin']}
{'_id': ObjectId('63d3d51eb0cd665042d3f8e0'), 'name': 'tong1', 'Gender': 'Male', 'Age': 21, 'Store': ['Nap x Warin', "NAP's Coffee & Roasters"]}


In [38]:
df1 = pd.DataFrame(cl)

In [39]:
df1 = df1.iloc[0:,[3,2,4]]

In [40]:
df1

,Age,Gender,Store
0,21,Male,[Nap x Warin]
1,21,Male,"[Nap x Warin, NAP's Coffee & Roasters]"


In [41]:
df.append(df1,ignore_index=True)

/var/folders/1m/8gfpxg2n77394ts5lp76hnw40000gn/T/ipykernel_41124/2746747452.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(df1,ignore_index=True)


,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
...,...,...,...
79,28,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
80,42,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
81,25,female,"NAP's Coffee & Roasters, sangob, SongSarn, Nap..."
82,21,Male,[Nap x Warin]


# raw data

In [42]:
df

,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
...,...,...,...
77,19,female,"NAP's Coffee & Roasters, Nap x Warin"
78,21,female,"NAP's Coffee & Roasters, sangob, SongSarn, LIF..."
79,28,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
80,42,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."


# setup data to list

In [316]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [43]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pickle

In [44]:
from pprint import pprint

In [45]:
data = pd.read_csv("data/Form.csv")
df = data.iloc[0:,[1,2,3]]
df

,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
...,...,...,...
77,19,female,"NAP's Coffee & Roasters, Nap x Warin"
78,21,female,"NAP's Coffee & Roasters, sangob, SongSarn, LIF..."
79,28,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."
80,42,female,"Nap x Warin, ROOF COFFEE, 11.11 Gallery and Co..."


In [46]:
df['Store'] = df['Store'].apply(lambda x: x.split(","))

In [48]:
# ADD USER TO RETRAIN
pipeline = [
    {
        '$lookup': {
            'from': "Transactions",
            'localField': "uid",
            'foreignField': "uid",
            'as': "combined_documents"
        }
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$unwind': "$combined_documents"
    },
    {
        '$match': { 'combined_documents.event': "view" }
    },
    {
        '$group': {
            '_id': "$_id",
            'name': { '$first': "$name" },
            'Gender': { '$first': "$gender" },
            'Age': { '$first': "$age" },
            'Store': {
                '$addToSet': "$combined_documents.Content"
            }
        }
    }
]


In [49]:
cl = list(collections.aggregate(pipeline))

In [53]:
df1 = pd.DataFrame(cl)
df1 = df1.iloc[0:,[3,2,4]]
df1["Gender"] = df1["Gender"].apply(lambda x:x.lower()) 

In [55]:
df1

,Age,Gender,Store
0,21,male,[Nap x Warin]
1,21,male,"[Nap x Warin, NAP's Coffee & Roasters]"


In [56]:
## DATA
df = df.append(df1,ignore_index=True)

/var/folders/1m/8gfpxg2n77394ts5lp76hnw40000gn/T/ipykernel_41124/1881829475.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1,ignore_index=True)


In [57]:
df

,Age,Gender,Store
0,20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
1,16,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA..."
2,22,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ..."
3,21,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop..."
4,19,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
...,...,...,...
79,28,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
80,42,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
81,25,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ..."
82,21,male,[Nap x Warin]


In [58]:
df1

,Age,Gender,Store
0,21,male,[Nap x Warin]
1,21,male,"[Nap x Warin, NAP's Coffee & Roasters]"


In [59]:
# setup age
df['age_group'] = df['Age'].apply(lambda x: '10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+')

In [60]:
df

,Age,Gender,Store,age_group
0,20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari...",10-20
1,16,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA...",10-20
2,22,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ...",21-40
3,21,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop...",21-40
4,19,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari...",10-20
...,...,...,...,...
79,28,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and...",21-40
80,42,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and...",41-60
81,25,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ...",21-40
82,21,male,[Nap x Warin],21-40


In [73]:
age_groups = df.groupby('age_group')
age_gender_rules = {}

# Run the Apriori algorithm and generate association rules for each age group and gender
for age_group, data in age_groups:
    for gender, gender_data in data.groupby('Gender'):
        # Use the TransactionEncoder to convert the data into a suitable format for the Apriori algorithm
        te = TransactionEncoder()
        te_ary = te.fit(gender_data['Store']).transform(gender_data['Store'])
        df_temp = pd.DataFrame(te_ary, columns=te.columns_)

        # Run the Apriori algorithm to find frequent item sets
        frequent_itemsets = apriori(df_temp, min_support=0.20, use_colnames=True)

        # Generate association rules from the frequent item sets
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1)
        # Store the association rules for each age group and gender in the dictionary
        age_gender_rules[(age_group, gender)] = rules

# with open('age_gender_rules1.pkl', 'wb') as f:
#     pickle.dump(age_gender_rules, f)


In [74]:
age_gender_rules

{('10-20',
  'female'):                                           antecedents  \
 0                                           ( Amarna)   
 1                                           ( sangob)   
 2                           ( MiND-K coffee and bake)   
 3                                (BalconyKiss Coffee)   
 4                           ( MiND-K coffee and bake)   
 5                                (BalconyKiss Coffee)   
 6                                           ( sangob)   
 7                              ( Bossa cafe,  sangob)   
 8                  (NAP's Coffee & Roasters,  sangob)   
 9                                           ( sangob)   
 10  ( LAVA  JAVA Coffee Roasters,  MiND-K coffee a...   
 11  ( LAVA  JAVA Coffee Roasters, BalconyKiss Coffee)   
 12      ( MiND-K coffee and bake, BalconyKiss Coffee)   
 13                          ( MiND-K coffee and bake)   
 14                               (BalconyKiss Coffee)   
 
                                           cons

In [367]:
with open('age_gender_rules1.pkl', 'rb') as f:
    age_gender_rules = pickle.load(f)




In [461]:
def recommend(user_age_group, user_gender):
    recommended_items = set()
    rules = age_gender_rules[(user_age_group, user_gender)].sort_values(by='confidence', ascending=False)
    for index, row in rules.head(15).iterrows():
        recommended_items.add(row["consequents"])
    recommended_items = list(recommended_items)
    recommended_items = [list(item) for item in recommended_items]
    return recommended_items


In [462]:
gender = input('Gender: ')
z = lambda x: '10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+'
age = z(int(input('Age: ')))
x = recommend(age, gender)



Gender:  male
Age:  20


In [463]:
re = set()
for a in x:
    da = set([ss.strip() for ss in a])
    re.update(da)
    

In [464]:
print(re)

{'Attaroast', 'BalconyKiss Coffee', 'GODFATHER COFFEE', 'sangob', 'Rogue Roasters', 'Saereesook', 'BaanHuakham Cafe & Farmstay', 'Yuanjai Cafe', 'Tree Cafe Rim Moon', 'Bossa cafe', 'Nap x Warin'}


In [465]:
list(re)

['Attaroast',
 'BalconyKiss Coffee',
 'GODFATHER COFFEE',
 'sangob',
 'Rogue Roasters',
 'Saereesook',
 'BaanHuakham Cafe & Farmstay',
 'Yuanjai Cafe',
 'Tree Cafe Rim Moon',
 'Bossa cafe',
 'Nap x Warin']